# FACE EMOTIONS DETECTOR


* Dataset link : https://drive.google.com/drive/folders/1tIQ-FFeHkB9ThT5GPPGiiKiBAUxyBQ8D?usp=sharing
* Haarcascade file link : https://github.com/tarun36rocker/Open-contributions/tree/master/haarcascades
* .h5 file ( you can run the already trained model if you please ) : https://github.com/tarun36rocker/Open-contributions/blob/master/model5.h5
* .md file that i made for this project : https://github.com/tarun36rocker/Open-contributions/blob/master/Tarun_OpenCV_Face-Emotions-Detector.md

# INFORMATION YOU MIGHT WANT TO KNOW

* I have SCRAPPED THE PICTURES myself so there might be few discrepancies
* This model is NOT THE MOST ACCURATE as there is not much data that i could work with and also because BingImageClassifier has a certain limit , i think because of the images webpage .

In [ ]:
# Used for scraping the pictures from BingImageCrawler
'''
from icrawler.builtin import BingImageCrawler
for keyword in ['human face happy']:
    crawler =  BingImageCrawler(
        parser_threads=2,
        downloader_threads=4,
        storage={'root_dir': 'C:/Users/Tarun/Desktop/comp/projects/face_emotions_detector/dataset/test_set/happy'} #storage={root_dir':'C:/Users/Tarun/Desktop/comp/image1/{}'.format(keyword)}
    )
    crawler.crawl(keyword=keyword, max_num=10, min_size=(200, 200)) '''

In [ ]:
from keras.models import Sequential #initialise the neural network as its a sequence of layers
from keras.layers import Conv2D #import convolutional layers
from keras.layers import MaxPooling2D #import pooling layers which have max function
from keras.layers import Flatten #import flatten layer
from keras.layers import Dense #import dense layer
from keras.preprocessing.image import ImageDataGenerator #preprocesses images to prevent overfitting and enrichs our training set without increasing the number of images

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#used for getting different variations of our data (in this case pictures) as we only have a limited dataset
#ImageDataGenerator for training data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
#ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')


num_classes = len(training_set.class_indices)#to check if our model has been connected to the right folders and checking the indexes of the different emotions
print(training_set.class_indices) 

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Initialising the CNN
classifier = Sequential() # initialising the sequential layer

# Step 1 - Convolution layer
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu')) #32 is number of feature detectors having dimensions (3,3),input_shape is format into which images will be converted .. 3 channels (rgb) of 64 * 64 coloured pixels, relu function for non-colinearity

# Step 2 - Pooling .. reducing size of the feature maps
classifier.add(MaxPooling2D(pool_size = (2, 2))) #2,2 is used so that we dont loose information and also be precise in where features are detected

# Adding a second convolutional layer of convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening layer
classifier.add(Flatten())

# Step 4 - Full connection / dense layers
classifier.add(Dense(units = 128, activation = 'relu')) #no. of nodes = 128 ( in power of 2 ), number around 100 is good , relu activation function is used
classifier.add(Dense(units = num_classes, activation = 'softmax')) # softmax activation function is used here to obtain the final output or the probabilites

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) # adam  stochastic gradient algorithm used

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Part 2 - Fitting the CNN to the images

classifier.fit_generator(training_set,
                         steps_per_epoch = 1000,
                         epochs = 7, #reduce number of epochs because cpu times out, originally 25
                         validation_data = test_set)

# Saving model to disk
classifier.save('model5.h5')

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#checking if our model works on a single picture
from keras.preprocessing import image
from keras import models
import numpy as np

model = models.load_model('model5.h5') #loading the trained model
test_image=image.load_img('dataset/single_pred/1.jpg',target_size = (64, 64)) #testing on a single picture
test_image=image.img_to_array(test_image) #converts it into 3d array
test_image=np.expand_dims(test_image,axis=0)
prediction=model.predict([test_image]) #finding the prediction by inputting the array into the model

count=0
for i in range(len(prediction[0])):
    if(prediction[0][i]==0):
        count+=1
    elif(prediction[0][i]==1):
        break
print(prediction[0])        
print(count)

In [ ]:
import numpy as np
import cv2 
from keras import models
from PIL import Image
from keras.preprocessing import image

model = models.load_model('model5.h5') #loading the model
cap = cv2.VideoCapture(0) #getting the object from the webcam ( default webcam input is at 0)
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')#using haarcascade_frontalface to detect the face

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#main function to detect the face and the emotions its portraying
def detect_face(img,frame_count):
    face_img = img.copy() #getting a copy of the frame
    face_rects = face_cascade.detectMultiScale(face_img) #detecting the face from the frame
    for (face_x,face_y,w,h) in face_rects:
        roi = frame[face_y:face_y+h, face_x:face_x+w] #getting the region of interest of the face
        roi = cv2.resize(roi, (64,64)) #resizing to suit our model
        cv2.imwrite('frame.jpg', roi) #saving as image so as to make changing of sizes and changing to arrays easier for us
        roi=image.load_img('frame.jpg',target_size = (64, 64))
        img_array=image.img_to_array(roi) #converts it into 3d array
        img_array=np.expand_dims(img_array,axis=0)
        prediction=model.predict([img_array]) #predicting by passing array into the model

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        count=0
        for i in range(len(prediction[0])): #getting the predicition value from the array
            if(prediction[0][i]==0):
                count+=1
            elif(prediction[0][i]==1):
                break       
        #Checking the predicted emotion based on our prediction        
        #BGR COLOR CODE
        if(count==0.0): 
            prediction="Angry" 
            color=(0,0,255)
        elif(count==1.0):
            prediction="Disgust"
            color=(0,255,0)
        elif(count==2.0):
            prediction="Fear"
            color=(255,0,0)
        elif(count==3.0):
            prediction="Happy"
            color=(0,255,255)
        elif(count==4.0):
            prediction="Sad"
            color=(139,0,0)
        else:
            prediction="Dont know!"
            color=(156,121,135)
            
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------  
        #writing our predictions on our live frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.rectangle(img, (face_x,face_y), (face_x+w,face_y+h), (255,255,255), 2)
        if(frame_count%2==0):
            cv2.putText(img,prediction,(200,130), font, 1, color, 2, cv2.LINE_AA) 
    return img

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

frame_count=0     
while True: 
    #getting frames from webcam
    ret, frame = cap.read(0)
    #passing frames to the emotions detection function
    frame = detect_face(frame,frame_count)
    frame_count+=1
    cv2.imshow('Emotions detection', frame)#final outputting of the frame 
    c = cv2.waitKey(1) 
    if c == 27: 
        break      
cap.release() 
cv2.destroyAllWindows()